In [19]:
# import find_context from baselines/bm25/bm25.py
import sys
sys.path.append('baselines')
from bm25 import rank_contexts

import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
test_df = pd.read_parquet('/scratch/tsoares/wikidumps/simplewiki-NS0-20230901/ml_data/test/')
test_df

,source_title,source_lead,target_title,target_lead,link_context,source_section,context_span_start_index,context_span_end_index,context_sentence_start_index,context_sentence_end_index,...,link_context_neg_17,label_neg_17,neg_type_neg_17,noise_strategy_neg_17,source_section_neg_17,link_context_neg_18,label_neg_18,neg_type_neg_18,noise_strategy_neg_18,source_section_neg_18
0,2007,2007 (MMVII) was a common year starting on Mon...,November 24,November 24 is the 328th day of the year in th...,"1955)\nNovember 20 – Ian Smith, 8th Prime Mini...",Deaths,139,231,139,225,...,"Oakland is a town in Orange County, Florida, U...",0,easy_replace_context,mask_mention,Lead,Andrey Kurkov (born 1961) is a Ukrainian write...,0,easy_replace_context,mask_mention,Lead
1,April 2,April 2 is the 92nd day of the year in the Gre...,Motor neurone disease,"Motor neurone disease, also known as amyotroph...","1983 - Felix Borja, Ecuadorean footballer\n198...",Births,244,299,244,300,...,Ansonia is a city in Connecticut in the United...,0,easy_replace_context,mask_sentence,Lead,Observances International Children's Book Day ...,0,hard_replace_context,mask_sentence,Observances
2,Jacinda Ardern,Dame Jacinda Kate Laurell Ardern is a New Zea...,Inauguration of Donald Trump,The inauguration of Donald Trump as the 45th p...,"She again finished second, but increased her o...","Member of Parliament, 2008–2023",324,935,610,773,...,The Jacqueline Kennedy Garden is located at th...,0,easy_replace_context,mask_span,Lead,Relations between the two countries had not be...,0,hard_replace_context,mask_span,"Prime Minister, 2017–2023"
3,February 20,February 20 is the 51st day of the year in the...,Larry King Live,Larry King Live was an American television tal...,"1987 – Unabomber: In Salt Lake City, in the US...",Events,374,487,374,488,...,"1899 – Cornelius Vanderbilt Whitney, American ...",0,hard_replace_context,mask_sentence,Births,February 20 is the 51st day of the year in the...,0,hard_replace_context,mask_sentence,Lead
4,July 13,July 13 is the 194th day of the year in the Gr...,2008,2008 (MMVIII) was a leap year starting on Tues...,"1907)\n2004 – Carlos Kleiber, Austrian conduct...",Deaths,103,166,103,160,...,Holidays and observances Kiribati – Independen...,0,hard_replace_context,no_mask,Holidays and observances,Related pages cant Precentor Kantor Canter,0,easy_replace_context,no_mask,Related pages
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122579,May 4,May 4 is the 124th day of the year in the Greg...,Tornado,A tornado is a tube of violently spinning air ...,"1998 - ""Unabomber"" Theodore Kaczynski is given...",Events,371,424,371,425,...,"1649 - Chhatrasal, Indian warrior (d. 1731) 16...",0,hard_replace_context,mask_sentence,Births,Arrast-Larrebieu is a commune of the Pyrénées-...,0,easy_replace_context,mask_sentence,Lead
122580,"Bisbee, Arizona","Bisbee is a town in Cochise County, Arizona, U...","Cochise County, Arizona",Cochise County is a county in the southeastern...,"Bisbee is a town in , Arizona, USA. It is abou...",Lead,0,215,0,49,...,Actors Yura Chuchunov as Chuk Andrei Chilikin ...,0,easy_replace_context,mask_mention,Actors,Related pages List of knots,0,easy_replace_context,mask_mention,Related pages
122581,February 18,February 18 is the 49th day of the year in the...,1911,1911 (MCMXI) was a common year starting on Sun...,"1885 - The Adventures of Huckleberry Finn, by ...",Events,395,469,395,470,...,"Fernando Gómez Colomer (born 11 September, 196...",0,easy_replace_context,mask_mention,Lead,Dissection Coxeter states that every parallel-...,0,easy_replace_context,mask_mention,Dissection
122582,January 16,January 16 is the 16th day of the year in the ...,Boxing,Boxing is one of the world's oldest combat spo...,"1959 – Sade, Nigerian singer\n1959 - Juanita B...",Births,218,255,218,256,...,2016 - A terrorist attack is carried out on a ...,0,hard_replace_context,no_mask,Events,"Freberg was born on August 7, 1926 in Pasadena

# Apply BM25

In [3]:
true_rank = []
target_titles = test_df['target_title'].tolist()
target_leads = test_df['target_lead'].tolist()
contexts = [[] for _ in range(len(target_titles))]
for column in test_df:
    if 'link_context' in column:
        for i, context in enumerate(test_df[column].tolist()):
            contexts[i].append(context)

In [4]:
for contexts, title, lead in tqdm(zip(contexts, target_titles, target_leads), total = len(target_titles)):
    scores = rank_contexts(contexts, title, lead)
    position = 1
    for score in scores[1:]:
        if score > scores[0]:
            position += 1
    true_rank.append(position)

100%|██████████| 122584/122584 [05:37<00:00, 363.18it/s]


In [5]:
test_df['bm25_rank'] = true_rank

## Compute MRR@k, Hits@k, NDCG@k

In [17]:
mrr_at_k = {'1': 0, '5': 0, '10': 0}
for rank in tqdm(test_df['bm25_rank'].tolist()):
    if rank <= 10:
        for k in mrr_at_k.keys():
            if rank <= int(k):
                mrr_at_k[k] += (1 / rank) / len(test_df)

100%|██████████| 122584/122584 [00:00<00:00, 544747.01it/s]


In [18]:
hits_at_k = {'1': 0, '5': 0, '10': 0}
for rank in tqdm(test_df['bm25_rank'].tolist()):
    if rank <= 10:
        for k in hits_at_k.keys():
            if rank <= int(k):
                hits_at_k[k] += 1 / len(test_df)

100%|██████████| 122584/122584 [00:00<00:00, 465450.31it/s]


In [20]:
ndcg_at_k = {'1': 0, '5': 0, '10': 0}
for rank in tqdm(test_df['bm25_rank'].tolist()):
    if rank <= 10:
        for k in ndcg_at_k.keys():
            if rank <= int(k):
                ndcg_at_k[k] += (1 / np.log2(rank + 1)) / len(test_df)

100%|██████████| 122584/122584 [00:00<00:00, 246146.29it/s]


In [23]:
print('BM25 Ranking Results')
print('Hits')
print(f'\tHits@1: {hits_at_k["1"]*100:.2f}%')
print(f'\tHits@5: {hits_at_k["5"]*100:.2f}%')
print(f'\tHits@10: {hits_at_k["10"]*100:.2f}%')
print('MRR')
print(f'\tMRR@1: {mrr_at_k["1"]*100:.2f}%')
print(f'\tMRR@5: {mrr_at_k["5"]*100:.2f}%')
print(f'\tMRR@10: {mrr_at_k["10"]*100:.2f}%')
print('NDCG')
print(f'\tNDCG@1: {ndcg_at_k["1"]*100:.2f}%')
print(f'\tNDCG@5: {ndcg_at_k["5"]*100:.2f}%')
print(f'\tNDCG@10: {ndcg_at_k["10"]*100:.2f}%')

BM25 Ranking Results
Hits
	Hits@1: 27.33%
	Hits@5: 48.30%
	Hits@10: 60.76%
MRR
	MRR@1: 27.33%
	MRR@5: 34.89%
	MRR@10: 36.52%
NDCG
	NDCG@1: 27.33%
	NDCG@5: 38.22%
	NDCG@10: 42.22%


## Get Discretized Statistics

In [30]:
lengths = {'no_mask': len(test_df[test_df['noise_strategy'] == 'no_mask']),
           'mask_mention': len(test_df[test_df['noise_strategy'] == 'mask_mention']),
           'mask_sentence': len(test_df[test_df['noise_strategy'] == 'mask_sentence']),
           'mask_span': len(test_df[test_df['noise_strategy'] == 'mask_span'])}

In [33]:
noise_mrr_at_k = {'no_mask': {'1': 0, '5': 0, '10': 0},
                  'mask_mention': {'1': 0, '5': 0, '10': 0},
                  'mask_sentence': {'1': 0, '5': 0, '10': 0},
                  'mask_span': {'1': 0, '5': 0, '10': 0}}
for rank, noise in tqdm(zip(test_df['bm25_rank'].tolist(), test_df['noise_strategy'].tolist()), total=len(test_df)):
    if rank <= 10:
        for k in noise_mrr_at_k[noise].keys():
            if rank <= int(k):
                noise_mrr_at_k[noise][k] += (1 / rank) / lengths[noise]

100%|██████████| 122584/122584 [00:00<00:00, 692163.89it/s]


In [34]:
noise_hits_at_k = {'no_mask': {'1': 0, '5': 0, '10': 0},
                   'mask_mention': {'1': 0, '5': 0, '10': 0},
                   'mask_sentence': {'1': 0, '5': 0, '10': 0},
                   'mask_span': {'1': 0, '5': 0, '10': 0}}
for rank, noise in tqdm(zip(test_df['bm25_rank'].tolist(), test_df['noise_strategy'].tolist()), total=len(test_df)):
    if rank <= 10:
        for k in noise_hits_at_k[noise].keys():
            if rank <= int(k):
                noise_hits_at_k[noise][k] += 1 / lengths[noise]

100%|██████████| 122584/122584 [00:00<00:00, 718381.92it/s]


In [35]:
noise_ndcg_at_k = {'no_mask': {'1': 0, '5': 0, '10': 0},
                   'mask_mention': {'1': 0, '5': 0, '10': 0},
                   'mask_sentence': {'1': 0, '5': 0, '10': 0},
                   'mask_span': {'1': 0, '5': 0, '10': 0}}
for rank, noise in tqdm(zip(test_df['bm25_rank'].tolist(), test_df['noise_strategy'].tolist()), total=len(test_df)):
    if rank <= 10:
        for k in noise_ndcg_at_k[noise].keys():
            if rank <= int(k):
                noise_ndcg_at_k[noise][k] += (1 / np.log2(rank + 1)) / lengths[noise]

100%|██████████| 122584/122584 [00:00<00:00, 291475.96it/s]


In [36]:
print('BM25 Discretized Ranking Results')
print('Hits')
print('\tNo Mask')
for k in noise_hits_at_k['no_mask'].keys():
    print(f'\t\tHits@{k}: {noise_hits_at_k["no_mask"][k]*100:.2f}%')
print('\tMask Mention')
for k in noise_hits_at_k['mask_mention'].keys():
    print(f'\t\tHits@{k}: {noise_hits_at_k["mask_mention"][k]*100:.2f}%')
print('\tMask Sentence')
for k in noise_hits_at_k['mask_sentence'].keys():
    print(f'\t\tHits@{k}: {noise_hits_at_k["mask_sentence"][k]*100:.2f}%')
print('\tMask Span')
for k in noise_hits_at_k['mask_span'].keys():
    print(f'\t\tHits@{k}: {noise_hits_at_k["mask_span"][k]*100:.2f}%')
print('MRR')
print('\tNo Mask')
for k in noise_mrr_at_k['no_mask'].keys():
    print(f'\t\tMRR@{k}: {noise_mrr_at_k["no_mask"][k]*100:.2f}%')
print('\tMask Mention')
for k in noise_mrr_at_k['mask_mention'].keys():
    print(f'\t\tMRR@{k}: {noise_mrr_at_k["mask_mention"][k]*100:.2f}%')
print('\tMask Sentence')
for k in noise_mrr_at_k['mask_sentence'].keys():
    print(f'\t\tMRR@{k}: {noise_mrr_at_k["mask_sentence"][k]*100:.2f}%')
print('\tMask Span')
for k in noise_mrr_at_k['mask_span'].keys():
    print(f'\t\tMRR@{k}: {noise_mrr_at_k["mask_span"][k]*100:.2f}%')
print('NDCG')
print('\tNo Mask')
for k in noise_ndcg_at_k['no_mask'].keys():
    print(f'\t\tNDCG@{k}: {noise_ndcg_at_k["no_mask"][k]*100:.2f}%')
print('\tMask Mention')
for k in noise_ndcg_at_k['mask_mention'].keys():
    print(f'\t\tNDCG@{k}: {noise_ndcg_at_k["mask_mention"][k]*100:.2f}%')
print('\tMask Sentence')
for k in noise_ndcg_at_k['mask_sentence'].keys():
    print(f'\t\tNDCG@{k}: {noise_ndcg_at_k["mask_sentence"][k]*100:.2f}%')
print('\tMask Span')
for k in noise_ndcg_at_k['mask_span'].keys():
    print(f'\t\tNDCG@{k}: {noise_ndcg_at_k["mask_span"][k]*100:.2f}%')

BM25 Discretized Ranking Results
Hits
	No Mask
		Hits@1: 39.63%
		Hits@5: 62.79%
		Hits@10: 72.61%
	Mask Mention
		Hits@1: 27.34%
		Hits@5: 50.20%
		Hits@10: 64.97%
	Mask Sentence
		Hits@1: 18.05%
		Hits@5: 38.57%
		Hits@10: 53.33%
	Mask Span
		Hits@1: 5.95%
		Hits@5: 15.72%
		Hits@10: 27.20%
MRR
	No Mask
		MRR@1: 39.63%
		MRR@5: 48.27%
		MRR@10: 49.57%
	Mask Mention
		MRR@1: 27.34%
		MRR@5: 35.48%
		MRR@10: 37.42%
	Mask Sentence
		MRR@1: 18.05%
		MRR@5: 25.20%
		MRR@10: 27.13%
	Mask Span
		MRR@1: 5.95%
		MRR@5: 9.27%
		MRR@10: 10.73%
NDCG
	No Mask
		NDCG@1: 39.63%
		NDCG@5: 51.89%
		NDCG@10: 55.05%
	Mask Mention
		NDCG@1: 27.34%
		NDCG@5: 39.13%
		NDCG@10: 43.87%
	Mask Sentence
		NDCG@1: 18.05%
		NDCG@5: 28.50%
		NDCG@10: 33.24%
	Mask Span
		NDCG@1: 5.95%
		NDCG@5: 10.86%
		NDCG@10: 14.49%
